In [17]:
import pandas as pd
import glob
import math
import xlrd
import openpyxl
import re

# Define date interval quartals

def get_quartal(date):
    if date.month in [1, 2, 3]:
        return 'Q1'
    elif date.month in [4, 5, 6]:
        return 'Q2'
    elif date.month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

In [41]:
filepath = "/Users/graciaandriamiadana/Documents/Research Culture PGR project/data/Combined PGR studenship payments 2022-23.xlsx"
df = pd.read_excel(filepath, engine= 'openpyxl')
df.head(5)

,Student ID,Budget code,A/c Code,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,...,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,School,Start date,End date,Duration,Source,uplift to 1/10/2022 payment for cost of living increase
0,200343938,ECSY1A3R,3620,4515.5,NaN,NaN,4917,NaN,NaN,4917,...,NaN,4917,NaN,NaN,AIDD_EECS,2022-10-01 00:00:00,2026-09-30 00:00:00,48,BBSRC,401.5
1,220679211,ECSY1A2R,3620,NaN,NaN,6556,NaN,NaN,NaN,4917,...,NaN,4917,NaN,NaN,AIDD_EECS,2022-12-01 00:00:00,2026-11-30 00:00:00,48,BBSRC,NaN
2,200346331,BCCG1D3R,3620,5250,NaN,NaN,5750,NaN,NaN,5750,...,NaN,5750,NaN,NaN,BCI,2021-10-01 00:00:00,2024-09-30 00:00:00,36,CRUK + CoL Centre,500
3,200826925,IRM9045B,3620,4917,4917,NaN,4917,NaN,NaN,4917,...,NaN,4917,NaN,NaN,BCI,2022-10-01 00:00:00,2025-09-30 00:00:00,36,BCSC,0
4,200974709,PRN1040R,3620,4515.5,NaN,NaN,4917,NaN,NaN,4917,...,NaN,4917,NaN,NaN,BCI,2020-10-01 00:00:00,2024-09-30 00:00:00,48,BBSRC LIDo,401.5


In [42]:
# Reshape the DataFrame using pd.melt() to unpivot date columns
df_melted = pd.melt(df, id_vars=['Student ID', 'Budget code', 'A/c Code', 'School', 'Start date', 'End date', 'Duration', 'Source'],
                    var_name='Date', value_name='Amount')

# Convert 'Date' column to datetime format
df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')  # 'coerce' to handle any parsing errors

# # Convert non-numeric strings to NaN in 'Amount' columns
df_melted['Amount'] = pd.to_numeric(df_melted['Amount'], errors='coerce')

# Convert the "Source" column to strings 
df_melted["Source"] = df_melted["Source"].astype(str)

In [43]:
# Uncomment this block for dataset 2022-23
# but do not run for all other datasets

# Handle additional column ('uplift to 1/10/2022 payment for cost of living increase')
df_melted['uplift_amount'] = pd.to_numeric(df['uplift to 1/10/2022 payment for cost of living increase '], errors='coerce')

# Combine 'Amount' and 'uplift_amount' columns
df_melted['Total_Amount'] = df_melted['Amount'].fillna(0) + df_melted['uplift_amount'].fillna(0)

In [44]:
df_melted['School'] = df_melted['School'].fillna('')

# Perform case-insensitive replacements and transformations in the 'School' column of df_melted
replacements = {
    r'.*EECS.*': 'EECS',        # Replace entries containing "EECS" with "EECS"
    r'^.*Geog.*$': 'School of Geography',   # Replace entries containing "Geog" with "Geography"
    r'.*Blizard.*': 'Blizard Institute',  # Replace entries containing "Blizard" with "Blizard"
    r'.*BLIZARD.*': 'Blizard Institute',  # Replace entries containing "Blizard" with "Blizard"
    r'^.*SED.*$': 'SED' ,          # Replace entries containing "SED" with "SED"
    r'^.*IPHS.*$': 'IPHS' ,          # Replace entries containing "SED" with "SED"
    r'^.*SLLF.*$': 'SLLF' ,          
    r'^.*History.*$': 'School of History',           
    r'^.*Law.*$': 'School of Law',
}          


# Iterate over the dictionary and perform replacements
for pattern, replacement in replacements.items():
    mask = df_melted['School'].str.contains(pattern, case=True, regex=True, na=False)
    df_melted.loc[mask, 'School'] = replacement

# Remove rows where 'School' is an empty string
df_melted = df_melted[df_melted['School'] != '']


print(sorted(df_melted["School"].unique().tolist()))

['BCI', 'Blizard Institute', 'DCE_Maths', 'EECS', 'IPHS', 'IoD', 'IoD_BCI', 'SBBS', 'SBM', 'SED', 'SEF', 'SEMS', 'SLLF', 'SMS', 'SPCS', 'SPIR', 'School', 'School of Geography', 'School of History', 'School of Law', 'WHRI', 'WIPH']


In [46]:
# Read the Excel file containing the category mappings
mapping_file = '/Users/graciaandriamiadana/Documents/Research Culture PGR project/data/Sources classification.xlsx'
category_mapping = pd.read_excel(mapping_file)

# Initialize an empty dictionary for replacements
replacements = {}

# Iterate over each category and its sub-names to generate regex patterns
for category in category_mapping.columns:
    for name in category_mapping[category].dropna():
        # Create a regex pattern that matches the name within any text
        pattern = rf'.*{re.escape(name)}.*'
        replacements[pattern] = category

# replacements[r'^CDA supplemen.*'] = 'UKRI'
# replacements[r'^AIM CDT.*'] = 'UKRI'

# Additional entries grouped under "Charities"
charities_entries = [
    'HS Barlow Charitable Trust/ Paragraf',
    'Heart Research UK',
    'Horne Family Charitable Fdn',
    'Horne Family foundation',
    'Bowel Research UK',
    'Welcome',
    'Wellcome',
    'PWSA UK',
    'Prostate Cancer UK',
    'Stuart Hall Foundation (SHF).',
]

# Add these entries to the replacements dictionary under "Charities"
for entry in charities_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Charities'

# Additional entries grouped under "Industry"
industry_entries = [
    'AstraZeneca',
    'Artios Pharma',
    'Huawei',
    'Industrial top-up',
    'Industry',
    'Industry (Creative Assembly)',
    'LTA Cola',
    'META',
    'Meta Platforms, Inc.',
    'Microsoft',
    'COLA'
]

# Add these entries to the replacements dictionary under "Industry"
for entry in industry_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Industry'

# Additional entries grouped under "UKRI"
ukri_entries = [
    'CDA supplement',
    'AIM CDT',
]

# Add these entries to the replacements dictionary under "UKRI"
for entry in ukri_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'UKRI'

# Additional entries grouped under "Internal"
internal_entries = [
    'SBBS',
    'SPIR',
    'SPCS',
    'WIPH',
    'Wolfson',
    'Supervisor Project',
    'Supervisor project',
    'Faculty',
    'Faculty Match fund',
    'Faculty Match fund.  P/T rates from 01-Oct-22.',
    'Faculty match fund (50%)',
    'BAME studentship',
]

# Add these entries to the replacements dictionary under "Internal"
for entry in internal_entries:
    pattern = rf'.*{re.escape(entry)}.*'
    replacements[pattern] = 'Internal'

print(replacements)

for pattern, replacement in replacements.items():
    mask = df_melted['Source'].str.contains(pattern, case=True, regex=True, na=False)
    df_melted.loc[mask, 'Source'] = replacement


print(len(df_melted["Source"].unique().tolist()))
# print("\n\n")
(sorted(df_melted["Source"].unique().tolist()))


{'.*School.*': 'Internal', '.*QM\\ Principal.*': 'Internal', '.*QM.*': 'Internal', '.*Queen\\ Mary\\ U.*': 'Internal', '.*EPSRC.*': 'UKRI', '.*STFC.*': 'UKRI', '.*SBCS.*': 'Internal', '.*MRC\\ DTP.*': 'UKRI', '.*Barts\\ Charit.*': 'Charities', '.*UKRI.*': 'UKRI', '.*ESRC\\ LISS\\ DT.*': 'UKRI', '.*NERC\\ DTP.*': 'UKRI', '.*AHRC.*': 'UKRI', '.*BHF.*': 'Charities', '.*BCI.*': 'Internal', '.*SEMS.*': 'Internal', '.*School\\ of\\ Bu.*': 'Internal', '.*CRUK.*': 'Charities', '.*WHRI.*': 'Internal', '.*ESRC\\ 1\\+3.*': 'UKRI', ".*Principal's.*": 'Internal', '.*ESRC\\ LISS.*': 'UKRI', '.*MRC.*': 'UKRI', '.*Royal\\ Societ.*': 'Charities', '.*Department\\ f.*': 'Internal', '.*Medical\\ Rese.*': 'Charities', '.*EPSRC\\ DTP.*': 'UKRI', '.*Leverhulme\\ T.*': 'Charities', '.*School\\ \\(PAR\\).*': 'Internal', ".*QM\\ S'Ship.*": 'Internal', '.*MAT\\ grant\\ \\(E.*': 'UKRI', '.*QM\\ Studentsh.*': 'Internal', '.*SEF.*': 'Internal', '.*Life\\ Science.*': 'Internal', '.*ESRC\\ \\(LISS\\ D.*': 'UKRI', '.*

['50% LMK Thermosafe',
 '50% S&E matched funding',
 '?',
 'ANTRUK (Antibiotic Research UK)',
 'Acutus Medical',
 'Animal Free Research UK',
 'Aston Martin Formula One Team',
 'BCSC',
 'BCSC Alexandra Carrell',
 'Barry Reed Foundation',
 'Bela - 25%',
 'Belgian Rsch Inst, VITO',
 'Byte Dance',
 'CDT',
 'Carbon Numbers Ltd and match funded',
 'Charities',
 'DAACI',
 'DEFRA',
 'DSTL',
 'Deepmind',
 'Defence Science & Tech Lab',
 'Defence Science and Tech Lab, Gov UK',
 'Delphia',
 'EDA for Consulatancy Project',
 'EPRSC DTP CASE Conversion 2021',
 'ERC 2016 STG 715475',
 'EU',
 'Environmental Agency top up only',
 'Evonik Operations GmbH',
 'Flamin-GO',
 'Gamble Aware',
 'Gilead',
 'Government',
 'Industry',
 'Internal',
 'LISS - DTP (1+3)',
 'LISS - DTP (just +3)',
 'LISS DTP',
 'LiDo',
 'LiDo trs from RVC',
 'MRS Mini CDT 2020 (PreMMIR)',
 'Mini CDT',
 'Music Tribe - 25%',
 'NDA',
 'National Nuclear Lab Ltd',
 'National Polytechnic Inst',
 'One off payment for CDA top up back pay. Paid 

In [37]:
(df_melted.head(10))

,Student ID,Budget code,A/c Code,School,Start date,End date,Duration,Source,Date,Amount,uplift_amount,Total_Amount,Quartal
0,200343938,ECSY1A3R,3620,EECS,2022-10-01 00:00:00,2026-09-30 00:00:00,48,UKRI,2022-10-01,4515.50,401.50,4917.00,Q4
1,220679211,ECSY1A2R,3620,EECS,2022-12-01 00:00:00,2026-11-30 00:00:00,48,UKRI,2022-10-01,NaN,NaN,0.00,Q4
2,200346331,BCCG1D3R,3620,BCI,2021-10-01 00:00:00,2024-09-30 00:00:00,36,Charities,2022-10-01,5250.00,500.00,5750.00,Q4
3,200826925,IRM9045B,3620,BCI,2022-10-01 00:00:00,2025-09-30 00:00:00,36,BCSC,2022-10-01,4917.00,0.00,4917.00,Q4
4,200974709,PRN1040R,3620,BCI,2020-10-01 00:00:00,2024-09-30 00:00:00,48,UKRI,2022-10-01,4515.50,401.50,4917.00,Q4
5,180879283,CIFG1E9R,3620,BCI,2020-01-01 00:00:00,2023-12-31 00:00:00,48 months,Charities,2022-10-01,7014.25,0.00,7014.25,Q4
6,180771736,SMD1000B,3620,BCI,2019-10-01 00:00:00,2023-09-30 00:00:00,48,UKRI,2022-10-01,109.50,0.00,109.50,Q4
7,180771736,TBYB1C3R,3620,BCI,2019-10-01 00:00:00,2023-09-30 00:00:00,48,UKRI,2022-10-01,2257.75,200.75,2458.50,Q4
8,180771736,PRN1040R,3620,BCI,2019-10-01 00:00:00,2023-09-30 00:00:00,48,UKRI,2022-10-01,2257.75,200.75,2458.50,Q4
9,220673725,TBYY1A9R,3620,BCI,2022-10-01 00:00:00,2025-09-30 00:00:00,36,UKRI,2022-10-01,4515.50,401.50,4917.00,Q4


In [36]:
# Group by 'Budget code' and sum the 'Amount' for each budget code
budget_totals = df_melted.groupby('Budget code')['Total_Amount'].sum().reset_index()
budget_totals['Formatted Amount'] = budget_totals['Total_Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol
budget_totals.rename(columns={'Budget code': 'Budget code', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code
# print(budget_totals)

#### --- if requested by quartal: --- ####
# Apply custom date intervals to 'Date' column
df_melted['Quartal'] = df_melted['Date'].apply(get_quartal)

# Group by 'Budget code' and 'Quartal' and sum the 'Amount' for each combination
budget_quartal_totals = df_melted.groupby(['Budget code', 'Quartal'])['Total_Amount'].sum().reset_index()

# Display the total amounts per budget code and quartal
# print(budget_quartal_totals)

# Group by 'School' and sum the 'Amount' for each School
studentship_payments_by_school = df_melted.groupby('School')['Total_Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_school['Formatted Amount'] = studentship_payments_by_school['Total_Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_school.rename(columns={'School': 'School Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code
# print(studentship_payments_by_school)


# Group by 'Budget code' and 'Quartal' and sum the 'Amount' for each combination
studentship_payments_by_school_quartal_totals = df_melted.groupby(['School', 'Quartal'])['Total_Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_school_quartal_totals['Formatted Amount'] = studentship_payments_by_school_quartal_totals['Total_Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_school.rename(columns={'School': 'School Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code and quartal
# print(studentship_payments_by_school_quartal_totals)

# Group by 'Source' and sum the 'Amount' for each School
studentship_payments_by_source = df_melted.groupby('Source')['Total_Amount'].sum().reset_index()

# Format the 'Amount' column
studentship_payments_by_source['Formatted Amount'] = studentship_payments_by_source['Total_Amount'].apply(lambda x: f'£{x:,.2f}')  # Format amount as currency with pounds (£) symbol

# Rename columns for better display
studentship_payments_by_source.rename(columns={'Source': 'Source Name', 'Formatted Amount': 'Total Amount (£)'}, inplace=True)

# Display the total amounts per budget code
studentship_payments_by_source

,Source Name,Total_Amount,Total Amount (£)
0,50% LMK Thermosafe,9.031000e+03,"£9,031.00"
1,ANTRUK (Antibiotic Research UK),1.848967e+04,"£18,489.67"
2,Acutus Medical,7.703945e+03,"£7,703.94"
3,Animal Free Research UK,1.009136e+04,"£10,091.36"
4,Aston Martin Formula One Team,1.806200e+04,"£18,062.00"
5,BCSC,4.425300e+04,"£44,253.00"
6,Barry Reed Foundation,2.006950e+04,"£20,069.50"
7,Bela - 25%,5.017375e+03,"£5,017.38"
8,"Belgian Rsch Inst, VITO",1.966800e+04,"£19,668.00"
9,Byte Dance,0.000000e+00,£0.00


In [26]:
budget_totals

,Budget code,Total_Amount,Total Amount (£)
0,ART4020B,10309.750000,"£10,309.75"
1,ART4020B,344086.730000,"£344,086.73"
2,ART4040B,20069.500000,"£20,069.50"
3,ASTF1A9R,0.000000,£0.00
4,ASTF1B3R,14751.000000,"£14,751.00"
...,...,...,...
553,TMTL1D5R,24142.000000,"£24,142.00"
554,TMTP1A4R,6020.666667,"£6,020.67"
555,TMTW1A2R,3200.000000,"£3,200.00"
556,TMTY1A3R,20069.500000,"£20,069.50"


In [27]:
# studentship_payments_by_source[studentship_payments_by_source['Source Name'] == "LIDo (BBSRC)"]
studentship_payments_by_source

,Source Name,Total_Amount,Total Amount (£)
0,50% LMK Thermosafe,9031.000000,"£9,031.00"
1,ANTRUK (Antibiotic Research UK),18489.675000,"£18,489.67"
2,ARSACS,20062.000000,"£20,062.00"
3,Acutus Medical,7703.945000,"£7,703.94"
4,Animal Free Research UK,10091.365000,"£10,091.36"
...,...,...,...
168,Versus arthritis,11930.000000,"£11,930.00"
169,Zhoukou Tianjiukang Pharmaceutical Co. Ltd.,3750.000000,"£3,750.00"
170,iCASE,2760.000000,"£2,760.00"
171,n/a,169849.756667,"£169,849.76"


In [28]:
studentship_payments_by_school

,School Name,Total_Amount,Total Amount (£)
0,BCI,1.437206e+06,"£1,437,205.62"
1,Blizard Institute,1.144177e+06,"£1,144,176.92"
2,DCE_Maths,1.162625e+04,"£11,626.25"
3,EECS,2.928362e+06,"£2,928,361.71"
4,IPHS,1.497300e+04,"£14,973.00"
5,IoD,4.978480e+04,"£49,784.80"
6,IoD_BCI,2.050750e+04,"£20,507.50"
7,SBBS,1.320697e+06,"£1,320,697.31"
8,SBM,6.821622e+05,"£682,162.25"
9,SED,4.797182e+05,"£479,718.25"


In [29]:
studentship_payments_by_school_quartal_totals

,School,Quartal,Total_Amount,Formatted Amount
0,BCI,Q1,374000.533333,"£374,000.53"
1,BCI,Q2,336123.750000,"£336,123.75"
2,BCI,Q3,330234.000000,"£330,234.00"
3,BCI,Q4,396847.333333,"£396,847.33"
4,Blizard Institute,Q1,291210.881900,"£291,210.88"
...,...,...,...,...
83,WHRI,Q4,304288.947500,"£304,288.95"
84,WIPH,Q1,266884.265000,"£266,884.27"
85,WIPH,Q2,268415.407527,"£268,415.41"
86,WIPH,Q3,256744.450000,"£256,744.45"
